In [114]:
import os
os.chdir("..")

In [115]:
os.environ["AZURE_LANGUAGE_ENDPOINT"] = "https://datathon.cognitiveservices.azure.com/"
os.environ["AZURE_LANGUAGE_KEY"] = "abdeac0882324fb59b2cf327e84f783a"
os.environ["AZURE_TRANSLATION_ENDPOINT"] = "https://api.cognitive.microsofttranslator.com/"
os.environ["AZURE_TRANSLATION_KEY"] = "ed4486cc91334c13863a64530126be25"

In [116]:
DEBUG = True

In [117]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from typing import Any, Dict, List

In [118]:
from ner import sample_analyze_healthcare_entities
from translator import translate_en_to_de, translate_de_to_en
from summarizer import summarize

In [119]:
import re
import numpy as np
import pandas as pd
import copy

In [120]:
input_text_de = """Asthma bronchiale
Auslöser sind Blütenpollen oder Tierhaare genauso wie Kälte oder Belastung: Asthma bronchiale gehört zu den häufigsten chronischen Krankheiten in der Schweiz.
Asthma bronchiale ist eine chronische und entzündliche Erkrankung der Atemwege. Charakteristisch sind Atemnot, Hustenanfälle und pfeifende Atemgeräusche. Die Bronchien reagieren dabei auf verschiedene Reize wie Pollen oder Tierhaare, aber auch Kälte oder körperliche Belastung können asthmatische Beschwerden auslösen.
In der Schweiz ist ungefähr jedes 10. Kind von Asthma betroffen. Damit ist Asthma bei Kindern die häufigste chronische Krankheit überhaupt. Die Erkrankung bessert sich mit steigendem Alter jedoch spürbar, deshalb leidet nur noch jeder 14. Erwachsene an Asthma.
"""

input_text_en = """Bronchitis
Bronchitis is inflammation of the bronchi (large and medium-sized airways) in the lungs that causes coughing. Symptoms include coughing up sputum, wheezing, shortness of breath, and chest pain. Bronchitis can be acute or chronic.

Acute bronchitis usually has a cough that lasts around three weeks, and is also known as a chest cold. In more than 90% of cases the cause is a viral infection. These viruses may be spread through the air when people cough or by direct contact. A small number of cases are caused by a bacterial infection such as Mycoplasma pneumoniae or Bordetella pertussis. Risk factors include exposure to tobacco smoke, dust, and other air pollution. Treatment of acute bronchitis typically involves rest, paracetamol (acetaminophen), and nonsteroidal anti-inflammatory drugs (NSAIDs) to help with the fever. 

"""

In [121]:
tokenizer_de_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model_de_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

tokenizer_en_de = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model_en_de = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

# summarizer_model_name = "ml6team/mt5-small-german-finetune-mlsum"
summarizer_model_name = "T-Systems-onsite/mt5-small-sum-de-en-v2"

summarizer_tokenizer = AutoTokenizer.from_pretrained(summarizer_model_name)
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(summarizer_model_name)

summarizer_tokenizer_en = AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_small_program_synthese_transfer_learning_finetune")
summarizer_model_en = AutoModel.from_pretrained("SEBIS/code_trans_t5_small_program_synthese_transfer_learning_finetune")

In [122]:
def translate(text: str, to_english: bool = True) -> str:
    if to_english:
        tokenized_text = tokenizer_de_en.prepare_seq2seq_batch([text], return_tensors='pt')
        translation = model_de_en.generate(**tokenized_text, max_length=1024)
        translated_text = tokenizer_de_en.batch_decode(translation, skip_special_tokens=True)[0]
    else:
        tokenized_text = tokenizer_en_de.prepare_seq2seq_batch([text], return_tensors='pt')
        translation = model_en_de.generate(**tokenized_text, max_length=1024)
        translated_text = tokenizer_en_de.batch_decode(translation, skip_special_tokens=True)[0]
    return translated_text

In [123]:
def translate_azure(text: str, target_lang='en'):
    if target_lang == 'en':
        return translate_de_to_en(text, os.environ["AZURE_TRANSLATION_KEY"], os.environ["AZURE_TRANSLATION_ENDPOINT"])
    else:
        return translate_en_to_de(text, os.environ["AZURE_TRANSLATION_KEY"], os.environ["AZURE_TRANSLATION_ENDPOINT"])


In [124]:
def abstract_summary(text: str, is_english: bool = True) -> str:
    if is_english:
        pass
        '''tokenized_text = summarizer_tokenizer_en.prepare_seq2seq_batch([text],max_length=1024, return_tensors='pt')
        output = summarizer_model_en.generate(**tokenized_text, max_length=1024)
        summary = summarizer_tokenizer_en.batch_decode(output, skip_special_tokens=True)[0]'''
    else:
        tokenized_text = summarizer_tokenizer.prepare_seq2seq_batch([text],max_length=1024, return_tensors='pt')
        output = summarizer_model.generate(**tokenized_text, max_length=1024)
        summary = summarizer_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return summary

In [125]:
def extractive_summary_azure(text: str, max_sentence_count=4):
    return summarize(text, max_sentence_count)

In [126]:
def extractive_summary(text: str, is_english: bool = True) -> List[str]:
    if is_english:
        pass
        summary = [text]
    else:
        pass
        summary = [text]
    return summary

In [127]:
def ner(text: str, input_lang: str = 'en') -> Dict[str, Any]:
    out =  sample_analyze_healthcare_entities([text], return_docs=True, translated_lang=input_lang)
    if len(out['entities']) == 1:
        out = {
            'entities' : out['entities'][0],
            'entity_relations' : out['entity_relations'][0] if len(out['entity_relations']) > 0 else [],
        }
    return out

In [128]:
def ner_translate(ner_obj, target_lang):
    #translate text cat and entity category (disregard normalized text as not informative)
    transl_cols = ['text', 'category']
    cp = copy.deepcopy(ner_obj)
    for x in cp:
        x.pop('normalized_text', None)
        x['translated_lang'] = target_lang
        for c in transl_cols:
            # split by Capitalization
            inp = " ".join(re.split('(?=[A-Z])', x[c]))
            out = translate_azure(inp, target_lang)
            if c == 'category':
                out = out.title().replace(" ", "")
            x[f'{c}_translated'] = out
    
    return cp

## German to English pipeline

In [129]:
## right, top (summarize, translate)
out_text_en = translate(input_text_de)
out_text_azure_en = translate_azure(input_text_de, target_lang='en')
# abstract_summary_en = abstract_summary(out_text_en) # NOT PRINTED
extractive_summary_en = extractive_summary(out_text_en)
extractive_summary_azure_en = extractive_summary_azure(out_text_en, max_sentence_count=4)
# use azure translated output for NER
ner_in_de = ner(input_text_de, input_lang = 'de')
ner_out_azure_en_in_de = ner(out_text_azure_en, input_lang = 'en') # not printed

# comparison summarized ger vs summarized english of NER
summ_input_text_de = extractive_summary(input_text_de)
ner_in_summ_de = ner(summ_input_text_de[0], input_lang = 'de')
ner_in_summ_out_azure_en_in_de = ner(extractive_summary_azure_en, input_lang = 'en') # TODO: SHOULD USE ACTUAL ENG TEXT THAT GETS SUMMARIZED



# top NOT PRINTED (summarize without translate)
# TODO: for comparison of sum-trans vs trans-sum
abstract_summary_de = abstract_summary(input_text_de, is_english = False)
extractive_summary_de = extractive_summary(input_text_de, is_english = False)
#abstract_summary_azure_de = abstract_summary(input_text_azure_de, is_english = False)
#extractive_summary_azure_de = extractive_summary(input_azure_text_de, is_english = False)


## English to German pipeline

In [134]:
## TODO:
# left, top
_input_text_en = """Bronchitis is inflammation of the bronchi (large and medium-sized airways) in the lungs that causes coughing. Symptoms include coughing up sputum, wheezing, shortness of breath, and chest pain. Bronchitis can be acute or chronic.
Acute bronchitis usually has a cough that lasts around three weeks, and is also known as a chest cold. In more than 90% of cases the cause is a viral infection.[4] These viruses may be spread through the air when people cough or by direct contact.
A small number of cases are caused by a bacterial infection such as Mycoplasma pneumoniae or Bordetella pertussis.
Risk factors include exposure to tobacco smoke, dust, and other air pollution. Treatment of acute bronchitis typically involves rest, paracetamol (acetaminophen), and nonsteroidal anti-inflammatory drugs (NSAIDs) to help with the fever."""

input_text_de = translate(_input_text_en)
input_text_azure_en = translate_azure(_input_text_en, target_lang='de')
# abstract_summary_en = abstract_summary(input_text_en)
extractive_summary_en = extractive_summary(_input_text_en)
extractive_summary_azure_en = extractive_summary_azure(_input_text_en, max_sentence_count=4)
ner_en = ner(input_text_de)

# SUMMARIZE 
# top
abstract_summary_de = abstract_summary(_input_text_en, is_english = False)
extractive_summary_de = extractive_summary(_input_text_en, is_english = False)
# abstract_summary_azure_de = abstract_summary(input_text_azure_de, is_english = False)
#extractive_summary_azure_de = extractive_summary(input_azure_text_de, is_english = False)
ner_de = ner(_input_text_en, input_lang = 'en')

In [133]:
print(input_text_de)

Bronchitis is inflammation of the bronchi (large and medium-sized airways) in the lungs that causes coughing. Symptoms include cutting up sputum, whiezing, shortness of breath, and chest pain. Bronchitis can be acute or chronic. Acute bronchitis usually has a cough that lasts around three weeks, and is also known as a Chest cold. In more than 90% of cases the cause is a viral infection.[4] These viruses may be spread through the air when people cough or by direct contact.[6] A small number of cases are caused by a bacterial infection such as Mycoplasma pneumoniae or Bordetella pertussis. Risks include exposure to tobacco and bone disease.


In [ ]:
def transform_ner(ner, input_lang, unique=False, verbose=False):
    if input_lang == 'en':
        cols = ['text', 'normalized_text', 'category']
    else:
        cols = ['text', 'category']
    
    if verbose:
        print('_Keys:', ner['entities'][0].keys())
    out = [{c: getattr(x, c) for c in cols } for x in ner['entities']]

    if unique:
        unq = {}
        for x in out:
            list_out = [x[c] for c in cols if x[c] is not None]
            #print(list_out)
            unq[f'{" ".join(list_out)}'] = x
            out = list(unq.values())
    print(out)
    print('______', end="\n\n")
    #if input_lang != 'en':
    #   print(ner)
    return out

# transformed NER german inpu
tf_ner_in_de = transform_ner(ner_in_de, input_lang='de')
# transformed NER english translated out
tf_ner_out_azure_en_in_de = transform_ner(ner_out_azure_en_in_de, input_lang='en', unique=True)
# translate to german transformed NER english output
translated_tf_ner_out_azure_en_in_de = ner_translate(tf_ner_out_azure_en_in_de, 'de')
print(translated_tf_ner_out_azure_en_in_de)
#transform_ner(translated_ner_out_azure_en_in_de, input_lang='en', unique=True)


print(ner_out_azure_en_in_de['entities'][0].keys())
print(ner_out_azure_en_in_de['entity_relations'][0].keys())

print('SUMMARIZED________________')

tf_ner_in_summ_de = transform_ner(ner_in_summ_de, input_lang='de')

tf_ner_in_summ_out_azure_en_in_de = transform_ner(ner_in_summ_out_azure_en_in_de, input_lang='en', unique=True)
# translate to german transformed NER english output
translated_tf_ner_in_summ_out_azure_en_in_de = ner_translate(tf_ner_in_summ_out_azure_en_in_de, 'de')
print(translated_tf_ner_in_summ_out_azure_en_in_de)







In [ ]:
print("Now we print the summary in English")
print()
#print(extractive_summary_azure_en)
print()
print("Now we translate the summary to German")
print()
translate_de_eng_summary = translate_azure(extractive_summary_azure_en, target_lang='de')
print(translate_de_eng_summary)
print()
print("NER on English summary")
print()
ner_on_eng_summary = ner(extractive_summary_azure_en, input_lang='en')
#print(ner_on_eng_summary)
print()
tf_ner_eng_summary = transform_ner(ner_on_eng_summary, input_lang='en', unique=True)
#print(tf_ner_eng_summary)
print()
trans_ner_eng_summary_to_de = ner_translate(tf_ner_eng_summary, 'de')
#print(trans_ner_eng_summary_to_de)
print()

list_of_de_ents = []

for item in trans_ner_eng_summary_to_de:
    list_of_de_ents.append(item['text_translated'])

print(list_of_de_ents)

count=0
for _item in list_of_de_ents:
    if _item in translate_de_eng_summary:
        count+=1
print()
print("Our EOP is {}".format(count/len(list_of_de_ents)))
        


In [ ]:
if DEBUG:
    print("DE INPUT:", input_text_de, end="\n\n")
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>", end="\n\n")

    def compare_texts(in_de, in_orig, in_azure, comp_txt):
        print(comp_txt)
        tpl = [re.split("[^0-9]\.", text.replace('\n', '')) for text in [in_de, in_orig, in_azure]]
        for de, orig, az in zip(*tpl):
            print('de', de)
            print('og', orig)
            print('az', az)
            print('__')
        
        print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>", end="\n\n")

    
    compare_texts(input_text_de, input_text_en, input_text_azure_en, "Comparison Orig vs Azure translation to english")
    '''
    print("Comparison Orig vs Azure translation to english")
    tpl = [text.replace('\n', '').split(".") for text in [input_text_de, input_text_en, input_text_azure_en]]
    for de, orig, az in zip(*tpl):
        print('de', de)
        print('og', orig)
        print('az', az)
        print('__')
            
    print("Input text translated to english:", end="\n\n")
    print(input_text_en, end="\n\n")
    
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>", end="\n")

    print("Input text translated by azure:", end="\n\n")
    print(input_text_azure_en, end="\n\n")
    print("--------------------------------", end="\n\n")
    
    
    print("Abstract summary in english:", end="\n\n")
    print(abstract_summary_en, end="\n\n")
    print("--------------------------------", end="\n\n")
    

    '''
    
    #compare_texts(input_text_de, extractive_summary_en[0], extractive_summary_azure_en, "Comparison Orig vs Azure summarization of translated")

    print("Input DE", end="\n")
    print(input_text_de, end="\n\n")

        
    print("Extractive summary in english:", end="\n")
    print(extractive_summary_en[0], end="\n\n")
    
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>", end="\n")
    
    print("Extractive summary by azure:", end="\n")
    print(extractive_summary_azure_en, end="\n\n")
    
    
    inde = "Der Patient klag ueber Kopfschmerzen. Es besteht keine erhoehte Koerpertemperatur. Der Puls ist normal, der Blutdruck jedoch niedrig."
    abstract_summary_de = abstract_summary(inde, is_english = False)
    print("Abstract summary in german:", end="\n\n")
    print(abstract_summary_de, end="\n\n")
    print("--------------------------------", end="\n\n")
    

    
    